
# Jour 3 — Algo Python pour Quant Dev

Ce notebook couvre les **algorithmes et structures de données** les plus demandés en entretien Quant Dev, avec pédagogie et exemples pratiques.  
Après ce notebook, tu sauras expliquer, coder, et adapter :

- BFS/DFS (parcours graphe, shortest path)
- Structures : queue, deque, heapq, Counter
- Dictionnaire trié (OrderedDict), set
- LRU cache (rappel rapide)
- Rolling statistics (calculs glissants en flux)
- Exos corrigés et quiz

---



## 1. BFS & DFS — Parcours de graphes

- **BFS (Breadth-First Search)** : explore couche par couche (niveau par niveau)
- **DFS (Depth-First Search)** : explore le plus loin possible avant de backtracker

**Applications :**
- Trouver le plus court chemin (si non pondéré)
- Explorer des connexions (order book, réseau, matching engine…)


In [ ]:

from collections import deque

# Graphe sous forme d'adjacence
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

# BFS (chemin de A vers F)
def bfs(graph, start, goal):
    queue = deque()
    queue.append((start, [start]))  # (noeud, chemin)
    visited = set()
    while queue:
        node, path = queue.popleft()
        if node == goal:
            return path
        if node in visited:
            continue
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                queue.append((neighbor, path + [neighbor]))
    return None

print("BFS A->F:", bfs(graph, 'A', 'F'))

# DFS
def dfs(graph, start, goal, path=None, visited=None):
    if path is None: path = [start]
    if visited is None: visited = set()
    visited.add(start)
    if start == goal:
        return path
    for neighbor in graph[start]:
        if neighbor not in visited:
            res = dfs(graph, neighbor, goal, path + [neighbor], visited)
            if res:
                return res
    return None

print("DFS A->F:", dfs(graph, 'A', 'F'))



**À retenir (oral)** :  
- BFS trouve toujours le chemin le plus court (non pondéré)
- DFS explore en profondeur, peut être plus rapide sur arbre peu large
- Utiliser `deque` pour la queue (O(1) pop/append)
- Pour un graphe pondéré : utiliser Dijkstra (avec `heapq`)

---



## 2. Structures de données : queue, deque, heap, Counter

### **queue/deque :**
- `deque` : file double entrée, très efficace (O(1) aux deux bouts)
- `queue.Queue` : version thread-safe (rare en quant dev)


In [ ]:

dq = deque([1,2,3])
dq.append(4)
dq.appendleft(0)
print("Deque :", dq)
dq.pop()
dq.popleft()
print("Après pop :", dq)



### **heapq :**
- Min-heap natif en Python (heapq)
- Utilisé pour extraire le plus petit (ou plus grand, si on stocke des -valeurs)


In [ ]:

import heapq
arr = [7, 3, 2, 9, 5]
heapq.heapify(arr)
print("Min-heap :", arr)
print("Extract min :", heapq.heappop(arr))
heapq.heappush(arr, 1)
print("Après push(1) :", arr)



### **Counter :**
- Compte les occurrences en O(n), très utile pour histogrammes, logs, parsing market data.


In [ ]:

from collections import Counter
arr = ['a', 'b', 'b', 'c', 'a', 'a']
c = Counter(arr)
print("Counter :", c)
print("Top 2 :", c.most_common(2))



### **OrderedDict / dict trié**

Depuis Python 3.7, dict natif est ordonné.  
`OrderedDict` reste utile pour LRU cache ou insertion/déplacement contrôlé.


In [ ]:

from collections import OrderedDict
od = OrderedDict()
od['A'] = 1
od['B'] = 2
od.move_to_end('A')
print("OrderedDict :", od)



## 3. LRU Cache (Least Recently Used)

- Structure : OrderedDict (ou dict+deque), capacity limitée
- Opérations O(1) pour get/put

**Code d'exemple (expliqué à l'oral) :**


In [ ]:

class LRUCache:
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity

    def get(self, key):
        if key not in self.cache:
            return -1
        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

lru = LRUCache(2)
lru.put(1, 10)
lru.put(2, 20)
print(lru.get(1))    # 10
lru.put(3, 30)       # Eviction de 2
print(lru.get(2))    # -1



**À l'oral** :  
> J'utilise OrderedDict pour garder l'ordre d'accès et pop l'élément le moins utilisé en O(1). C'est un pattern classique pour du cache en trading (ex: market data tick, last prices).



## 4. Rolling statistics sur un flux de données

Exemple : calcul de la moyenne glissante (rolling mean) d'un prix de marché (ex: mid price).

- En quant : utile pour signal, moving average crossover, volatility window, etc.



In [ ]:

import numpy as np
data = np.random.normal(100, 2, 100)
window = 5

# Rolling mean "à la main"
rolling_mean = [np.mean(data[i-window+1:i+1]) if i >= window-1 else np.nan for i in range(len(data))]
print("Derniers rolling mean :", rolling_mean[-5:])

# Avec pandas (pro)
import pandas as pd
df = pd.DataFrame({"price": data})
df["roll_mean"] = df["price"].rolling(window).mean()
print(df.tail(7))



**À retenir à l'oral :**
- Pour le streaming ou les calculs temps réel, une queue (deque) ou une formule récursive permet d'éviter de recalculer tout à chaque tick.



## 5. Exos classiques entretien

**1. Plus court chemin dans un graphe pondéré (Dijkstra)**  
(À connaître, mais version heapq/adjacency dict seulement)

**2. Trouver le 1er élément qui apparaît k fois dans un stream**

**3. Rolling max/min dans une fenêtre glissante**

**4. Détecter un cycle dans un graphe**

**Exemples pratiques en cellules à compléter :**


In [ ]:

import heapq
def dijkstra(graph, start):
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    heap = [(0, start)]
    while heap:
        d, node = heapq.heappop(heap)
        if d > dist[node]: continue
        for neighbor, weight in graph[node]:
            if dist[neighbor] > d + weight:
                dist[neighbor] = d + weight
                heapq.heappush(heap, (dist[neighbor], neighbor))
    return dist

# Graphe pondéré (exemple)
g2 = {'A': [('B',2), ('C',4)], 'B':[('C',1), ('D',7)], 'C':[('D',3)], 'D':[]}
print("Dijkstra A->", dijkstra(g2, 'A'))


In [ ]:

from collections import deque
def rolling_max(arr, k):
    q, res = deque(), []
    for i, x in enumerate(arr):
        while q and arr[q[-1]] < x:
            q.pop()
        q.append(i)
        if q[0] == i-k:
            q.popleft()
        if i >= k-1:
            res.append(arr[q[0]])
    return res

print("Rolling max sur [1,3,-1,-3,5,3,6,7], k=3 :", rolling_max([1,3,-1,-3,5,3,6,7], 3))



---
## 📑 **À retenir pour l'entretien :**

- **Savoir choisir la structure adaptée (dict, set, deque, heap, Counter)**
- **Savoir coder et expliquer BFS, DFS, Dijkstra (avec heapq)**
- **Rolling mean/max/min : idéal pour signaux, backtest rapide**
- **LRU cache, pattern d'optimisation classique en algo quant**
- **Structurer proprement son code pour passer de l'algo "papier" à un algo Python exploitable en prod**

---
